In [1]:
import math
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('../preprocess/csv_files/ALL_DATA.csv')
data.count()

In [ ]:
print(data["arousal"].max())  # arousal, control group
print(data["arousal"].min())
print(data["valence"].max())  # valence
print(data["valence"].min())

In [ ]:
data.groupby("dataset").count().rename(columns={"id": "amount"})["amount"]

In [ ]:
def get_quarter(valence, arousal):
    if valence >= 0 and arousal >= 0:
        return 1
    elif valence < 0 and arousal >= 0:
        return 2
    elif valence < 0 and arousal < 0:
        return 3
    else:
        return 4


data['quarter'] = data.apply(lambda row: get_quarter(row['valence'], row['arousal']), axis=1)

data.head()

In [ ]:
data.groupby(["quarter", "dataset"]).count().rename(columns={"id": "amount"})["amount"]

In [ ]:
data.groupby(["quarter"]).count()

In [ ]:
fig = plt.figure(figsize=(12, 12))
fig.tight_layout(pad=20)
ax = fig.add_subplot()

for index, row in data.iterrows():
    ax.scatter(row["valence"], row["arousal"], marker='o')

font = {'family': 'serif',
        'color': 'black',
        'weight': 'normal',
        'size': 16,
        }

ax.set_xlabel('Valence', fontdict=font)
ax.set_ylabel('Arousal', fontdict=font)

ax.set_xlim(-1.05, 1.05)
ax.set_ylim(-1.05, 1.05)

plt.grid()
plt.title("All data Valence-Arousal space graph", fontdict=font)

plt.savefig("plots/all_data_valence_arousal")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 12))
fig.tight_layout(pad=20)
ax = fig.add_subplot()

for index, row in data.iterrows():
    ax.scatter(row["valence"], row["arousal"], marker='o', color="lightblue")

font = {'family': 'serif',
        'color': 'black',
        'weight': 'normal',
        'size': 16,
        }

ax.set_xlabel('Valence', fontdict=font)
ax.set_ylabel('Arousal', fontdict=font)

r = 0.25
circle = plt.Circle((0, 0), r, color='r', fill=False, linewidth=2)
ax.add_patch(circle)

plt.plot([0, 0], [r, 1], color='r', linewidth=2)
plt.plot([r * math.cos(math.pi / 3), math.sqrt(3) / 3], [r * math.sin(math.pi / 3), 1], color='r',
         linewidth=2)
plt.plot([r * math.cos(math.pi / 6), 1], [r * math.sin(math.pi / 6), math.sqrt(3) / 3], color='r',
         linewidth=2)
plt.plot([r, 1], [0, 0], color='r', linewidth=2)
plt.plot([r * math.cos(math.pi / 6), 1], [-r * math.sin(math.pi / 6), -math.sqrt(3) / 3], color='r',
         linewidth=2)
plt.plot([r * math.cos(math.pi / 3), math.sqrt(3) / 3], [-r * math.sin(math.pi / 3), -1], color='r',
         linewidth=2)
plt.plot([0, 0], [-1, -r], color='r', linewidth=2)
plt.plot([-r * math.cos(math.pi / 3), -math.sqrt(3) / 3], [-r * math.sin(math.pi / 3), -1],
         color='r',
         linewidth=2)
plt.plot([-r * math.cos(math.pi / 6), -1], [-r * math.sin(math.pi / 6), -math.sqrt(3) / 3],
         color='r',
         linewidth=2)
plt.plot([-1, -r], [0, 0], color='r', linewidth=2)
plt.plot([-r * math.cos(math.pi / 6), -1], [r * math.sin(math.pi / 6), math.sqrt(3) / 3], color='r',
         linewidth=2)
plt.plot([-r * math.cos(math.pi / 3), -math.sqrt(3) / 3], [r * math.sin(math.pi / 3), 1], color='r',
         linewidth=2)

plt.text(-0.11, -0.005, "Neutral", fontsize=20, color="r")
plt.text(0.1, 0.95, "Excited", fontsize=20, color="r")
plt.text(0.65, 0.8, "Delighted", fontsize=20, color="r")
plt.text(0.8, 0.25, "Happy", fontsize=20, color="r")
plt.text(0.75, -0.25, "Content", fontsize=20, color="r")
plt.text(0.7, -0.8, "Relaxed", fontsize=20, color="r")
plt.text(0.15, -0.95, "Calm", fontsize=20, color="r")
plt.text(-0.35, -0.95, "Tired", fontsize=20, color="r")
plt.text(-0.9, -0.8, "Bored", fontsize=20, color="r")
plt.text(-1, -0.25, "Depressed", fontsize=20, color="r")
plt.text(-1, 0.25, "Frustrated", fontsize=20, color="r")
plt.text(-0.9, 0.8, "Angry", fontsize=20, color="r")
plt.text(-0.35, 0.95, "Tense", fontsize=20, color="r")

ax.set_xlim(-1.05, 1.05)
ax.set_ylim(-1.05, 1.05)

plt.grid()
plt.title("All data Valence-Arousal assigned to groups", fontdict=font)

plt.savefig("plots/all_data_valence_arousal_classified")
plt.show()

In [10]:
labels_map = {0: "neutral", 1: "content", 2: "relaxed", 3: "calm", 4: "tired", 5: "bored",
              6: "depressed", 7: "frustrated", 8: "angry", 9: "tense", 10: "excited",
              11: "delighted", 12: "happy"}

In [11]:
def calc_angle(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return np.rad2deg((ang1 - ang2) % (2 * np.pi))


data_result = []
for index, row in data.iterrows():
    angle = calc_angle((0, 0), (row.valence, row.arousal))

    if abs(row.valence) <= 0.25 and abs(row.arousal) <= 0.25:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "neutral", "category_id": 0})
    elif 0 <= angle < 30:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "content", "category_id": 1})
    elif 30 <= angle < 60:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "relaxed", "category_id": 2})
    elif 60 <= angle < 90:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "calm", "category_id": 3})
    elif 90 <= angle < 120:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "tired", "category_id": 4})
    elif 120 <= angle < 150:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "bored", "category_id": 5})
    elif 150 <= angle < 180:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "depressed", "category_id": 6})
    elif 180 <= angle < 210:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "frustrated", "category_id": 7})
    elif 210 <= angle < 240:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "angry", "category_id": 8})
    elif 240 <= angle < 270:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "tense", "category_id": 9})
    elif 270 <= angle < 300:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "excited", "category_id": 10})
    elif 300 <= angle < 330:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "delighted", "category_id": 11})
    elif 330 <= angle < 360:
        data_result.append({"dataset": row.dataset, "id": row.id, "valence": row.valence,
                            "arousal": row.arousal, "category": "happy", "category_id": 12})
    else:
        print("Angle not in range!")

In [ ]:
result = pd.DataFrame(data_result)
result.head()

In [ ]:
result.groupby(["category"]).count().rename(columns={"id": "amount"})["amount"]

In [14]:
# empty

In [ ]:
data = pd.read_csv('../preprocess/csv_files/EMOMADRID.csv')
data.count()

In [ ]:
data = pd.read_csv('../preprocess/csv_files/GAPED.csv')
data.count()

In [ ]:
data = pd.read_csv('../preprocess/csv_files/IAPS.csv')
data.count()

In [ ]:
data = pd.read_csv('../preprocess/csv_files/NAPS_H.csv')
data.count()

In [ ]:
data = pd.read_csv('../preprocess/csv_files/NAPS_H.csv')
data.count()

In [ ]:
data = pd.read_csv('../preprocess/csv_files/OASIS.csv')
data.count()

In [ ]:
data = pd.read_csv('../preprocess/csv_files/SFIP.csv')
data.count()


In [ ]:
data.describe()

In [ ]:
data_to_augment = pd.read_csv('../preprocess/csv_files/ALL_DATA.csv')

training_data = []
for index, row in data_to_augment.iterrows():
    image_name, img_ext = os.path.splitext(row.id)
    training_data.append(
        {"id": f"{row.dataset}_{image_name}_base{img_ext}", "label": row.category_id})
    training_data.append(
        {"id": f"{row.dataset}_{image_name}_detail{img_ext}", "label": row.category_id})
    training_data.append(
        {"id": f"{row.dataset}_{image_name}_edgeenhance{img_ext}", "label": row.category_id})
    training_data.append(
        {"id": f"{row.dataset}_{image_name}_bright{img_ext}", "label": row.category_id})
    training_data.append(
        {"id": f"{row.dataset}_{image_name}_bright2{img_ext}", "label": row.category_id})
    training_data.append(
        {"id": f"{row.dataset}_{image_name}_rotate90{img_ext}", "label": row.category_id})
    training_data.append(
        {"id": f"{row.dataset}_{image_name}_rotate180{img_ext}", "label": row.category_id})
    training_data.append(
        {"id": f"{row.dataset}_{image_name}_rotate270{img_ext}", "label": row.category_id})

training_data_result = pd.DataFrame(training_data)
training_data_result.head()

In [ ]:
training_data_result.count()

In [ ]:
data = pd.read_csv('../preprocess/csv_files/ALL_DATA.csv')
data.count()

In [ ]:
data[(data["dataset"] == "EMOMADRID") & (data["id"] == "EM0001.jpg")]

In [ ]:
training_data = []
validation_data = []
for index, row in data.iterrows():
    if random.random() < 0.2:
        validation_data.append({"id": f"{row.dataset}_{row.id}", "label": row.category_id})
    else:
        training_data.append({"id": f"{row.dataset}_{row.id}", "label": row.category_id})

training_data = pd.DataFrame(training_data)
training_data.count()

In [ ]:
validation_data = pd.DataFrame(validation_data)
validation_data.count()